In [1]:
import pandas as pd
import logging
import transformers
import torch
import numpy as np
from bert_score import score
import matplotlib.pyplot as plt
from matplotlib import rcParams
import itertools
# Suppress loading messages
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

/Users/gengliu/anaconda3/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (3.0.4)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(


In [2]:
# Matplotlib configuration
rcParams["xtick.major.size"] = 0
rcParams["xtick.minor.size"] = 0
rcParams["ytick.major.size"] = 0
rcParams["ytick.minor.size"] = 0
rcParams["axes.labelsize"] = "large"
rcParams["axes.axisbelow"] = True
rcParams["axes.grid"] = True

# Load data
baidu = pd.read_csv("../data/baidu_v5.csv")
ernie = pd.read_csv("../data/ernie_v6.csv")
qwen = pd.read_csv("../data/qwen_v6.csv")

In [3]:
# # Create similarity dataframe for ernie groups
# ernie_groups = ernie['group_English'].unique()
# qwen_groups = qwen['group_English'].unique()

# similarity_df = create_similarity_dataframe(ernie_groups)


NameError: name 'create_similarity_dataframe' is not defined

In [4]:
# Define function to create similarity dataframe (if grp1, grp2 is present, do not calculate grp2, grp1)
def create_similarity_dataframe(groups):
    combinations = [e for e in itertools.combinations(ernie_groups, 2)]
    return pd.DataFrame(combinations, columns=['group1', 'group2'])




In [5]:
ernie_groups = ernie['group_English'].unique()
similarity_df = create_similarity_dataframe(ernie_groups)

NameError: name 'similarity_df_qwen' is not defined

In [16]:
# Create similarity dataframe for ernie groups


ones = []
twos = []

# all the combinations for all the possible groups
for index, row in similarity_df.iterrows():
    print(index, end='\r')
    group1_responses = ernie[ernie['group_English'] ==
                             row['group1']]['json_response'].dropna().tolist()
    group2_responses = ernie[ernie['group_English'] ==
                             row['group2']]['json_response'].dropna().tolist()
    if group1_responses and group2_responses:
        elems = [e for e in itertools.product(
            group1_responses, group2_responses)]
        one = [e[0] for e in elems]
        two = [e[1] for e in elems]
        ones.extend(one)
        twos.extend(two)

# calculate each bert score only once! (you can map the scores later to "correspondences" df)
correspondences = pd.DataFrame({'ones': ones, 'twos': twos})
unique_correspondences = correspondences.drop_duplicates()

print(len(correspondences), len(unique_correspondences))



28882293 3230466


In [18]:
correspondences

,ones,twos
0,努力追求成功,努力
1,努力追求成功,坚强
2,努力追求成功,拼搏
3,努力追求成功,有毅力
4,努力追求成功,上进
...,...,...
28882288,有力量,勇于进取
28882289,有力量,团结一心
28882290,有力量,勤劳能干
28882291,有力量,善于学习


In [19]:
unique_ones = unique_correspondences.ones.tolist()
unique_twos = unique_correspondences.twos.tolist()

device = torch.device("mps")
P_scores, R_scores, F1_scores = score(
    unique_ones, unique_twos, lang='zh', verbose=True, device=device)

calculating scores...
computing bert embedding.


  0%|          | 0/36 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/50477 [00:00<?, ?it/s]

done in 2428.63 seconds, 1330.16 sentences/sec


In [20]:
# Assuming P_scores, R_scores, and F1_scores are tensors or similar structures, convert them to lists
P_scores_list = P_scores.tolist()
R_scores_list = R_scores.tolist()
F1_scores_list = F1_scores.tolist()

# Add these lists as new columns in the DataFrame
unique_correspondences['P_score'] = P_scores_list
unique_correspondences['R_score'] = R_scores_list
unique_correspondences['F1_score'] = F1_scores_list

/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_2101/2612561919.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_correspondences['P_score'] = P_scores_list
/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_2101/2612561919.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_correspondences['R_score'] = R_scores_list
/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_2101/2612561919.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [22]:
unique_correspondences.to_csv('ernie_bert_unique_correspondences.csv', index=False)

In [23]:
### Qwen 

In [28]:
# Create similarity dataframe for ernie groups


ones = []
twos = []

# all the combinations for all the possible groups
for index, row in similarity_df.iterrows():
    print(index, end='\r')
    group1_responses = qwen[qwen['group_English'] ==
                             row['group1']]['json_response'].dropna().tolist()
    group2_responses = qwen[qwen['group_English'] ==
                             row['group2']]['json_response'].dropna().tolist()
    if group1_responses and group2_responses:
        elems = [e for e in itertools.product(
            group1_responses, group2_responses)]
        one = [e[0] for e in elems]
        two = [e[1] for e in elems]
        ones.extend(one)
        twos.extend(two)

# calculate each bert score only once! (you can map the scores later to "correspondences" df)
correspondences = pd.DataFrame({'ones': ones, 'twos': twos})
unique_correspondences = correspondences.drop_duplicates()

print(len(correspondences), len(unique_correspondences))

40711855 2583861


In [29]:
unique_ones = unique_correspondences.ones.tolist()
unique_twos = unique_correspondences.twos.tolist()

device = torch.device("mps")
P_scores, R_scores, F1_scores = score(
    unique_ones, unique_twos, lang='zh', verbose=True, device=device)

calculating scores...
computing bert embedding.


  0%|          | 0/30 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/40373 [00:00<?, ?it/s]

done in 1910.27 seconds, 1352.61 sentences/sec


In [30]:
# Assuming P_scores, R_scores, and F1_scores are tensors or similar structures, convert them to lists
P_scores_list = P_scores.tolist()
R_scores_list = R_scores.tolist()
F1_scores_list = F1_scores.tolist()

# Add these lists as new columns in the DataFrame
unique_correspondences['P_score'] = P_scores_list
unique_correspondences['R_score'] = R_scores_list
unique_correspondences['F1_score'] = F1_scores_list

/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_2101/2612561919.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_correspondences['P_score'] = P_scores_list
/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_2101/2612561919.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_correspondences['R_score'] = R_scores_list
/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_2101/2612561919.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [32]:
# Add these lists as new columns in the DataFrame
unique_correspondences.to_csv("qwen_bert_unique_correspondences.csv", index=False)

In [33]:
## Baidu

In [11]:
baidu = baidu[baidu.suggestion_starts_with_query == True]

In [12]:
baidu.columns

Index(['category', 'group', 'group_English', 'querys', 'template_name',
       'response', 'group_in_response', 'suggestion_starts_with_query',
       'response_values', 'aliyun_sentiment', 'response_values_english',
       'word_embedding_response_values', 'combine_query_response',
       'value_alignment_response_qwen',
       'value_alignment_response_qwen_english',
       'value_alignment_response_ernie',
       'value_alignment_response_ernie_english'],
      dtype='object')

In [13]:
# Create similarity dataframe for ernie groups


ones = []
twos = []

# all the combinations for all the possible groups
for index, row in similarity_df.iterrows():
    print(index, end='\r')
    group1_responses = baidu[baidu['group_English'] ==
                             row['group1']]['response_values'].dropna().tolist()
    group2_responses = baidu[baidu['group_English'] ==
                            row['group2']]['response_values'].dropna().tolist()
    if group1_responses and group2_responses:
        elems = [e for e in itertools.product(
            group1_responses, group2_responses)]
        one = [e[0] for e in elems]
        two = [e[1] for e in elems]
        ones.extend(one)
        twos.extend(two)

# calculate each bert score only once! (you can map the scores later to "correspondences" df)
correspondences = pd.DataFrame({'ones': ones, 'twos': twos})
unique_correspondences = correspondences.drop_duplicates()

print(len(correspondences), len(unique_correspondences))

2271491 230247


In [14]:
unique_ones = unique_correspondences.ones.tolist()
unique_twos = unique_correspondences.twos.tolist()

device = torch.device("mps")
P_scores, R_scores, F1_scores = score(
    unique_ones, unique_twos, lang='zh', verbose=True, device=device)

calculating scores...
computing bert embedding.


  0%|          | 0/10 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3598 [00:00<?, ?it/s]

done in 177.37 seconds, 1298.10 sentences/sec


In [15]:
# Assuming P_scores, R_scores, and F1_scores are tensors or similar structures, convert them to lists
P_scores_list = P_scores.tolist()
R_scores_list = R_scores.tolist()
F1_scores_list = F1_scores.tolist()

# Add these lists as new columns in the DataFrame
unique_correspondences['P_score'] = P_scores_list
unique_correspondences['R_score'] = R_scores_list
unique_correspondences['F1_score'] = F1_scores_list

/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_42484/2612561919.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_correspondences['P_score'] = P_scores_list
/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_42484/2612561919.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_correspondences['R_score'] = R_scores_list
/var/folders/x1/rg74ttxx7bb0_2z1wxpb2z940000gn/T/ipykernel_42484/2612561919.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [16]:
# Add these lists as new columns in the DataFrame
unique_correspondences.to_csv(
    "baidu_bert_unique_correspondences.csv", index=False)

In [42]:
import os 
os.getcwd()

'/Users/gengliu/research/2024/stereotypes_in_search_engines-and-Chinese-LLMs/code'